In [1]:
import torch
import numpy as np
import pandas as pd
import uproot as ur

In [2]:
data = [[1.,2.], [3,4]]
x_data = torch.tensor(data)
print(x_data)
print(x_data.shape)
print(x_data.dtype)
print(x_data.device)

tensor([[1., 2.],
        [3., 4.]])
torch.Size([2, 2])
torch.float32
cpu


In [3]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
rand_tensor

tensor([[0.8781, 0.0341, 0.9347],
        [0.5923, 0.6541, 0.4529]])

In [4]:
print(rand_tensor[0])
print(rand_tensor[0,:-1])

tensor([0.8781, 0.0341, 0.9347])
tensor([0.8781, 0.0341])


In [70]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, signal_file: str, background_file: str, features: list[str], transform=None, target_transform=None) -> None:
        signal = ur.concatenate(signal_file, expressions=features, library="pd")
        signal["label"] = torch.ones(signal.shape[0], dtype=torch.float)

        background = ur.concatenate(background_file, expressions=features, library="pd")
        background["label"] = torch.zeros(background.shape[0], dtype=torch.float)

        self.data = pd.concat([signal,background])

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        event = self.data.iloc[idx]
        label = self.data.label.iloc[idx]
        if self.transform:
            event = self.transform(event)
            print(event)
        if self.target_transform:
            label = self.target_transform(label)
        return event, label

In [72]:
from torchvision.transforms import ToTensor, Lambda

features = ["B_MCORR", "B_ENDVERTEX_X"]
training_data = CustomDataset("./signal.root", "./background.root", features=features, transform=ToTensor)
# target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))

3


In [73]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [74]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [87]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(len(features), 2), # 28pixel * 28pixel images
            nn.ReLU(),
            nn.Linear(2, 2),
            nn.ReLU(),
            nn.Linear(2, 13), # 13 predictions
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [88]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=2, bias=True)
    (3): ReLU()
    (4): Linear(in_features=2, out_features=13, bias=True)
  )
)


In [89]:
X = torch.rand(5, 2, device=device) # 5 repeats
X

tensor([[0.6698, 0.6886],
        [0.9259, 0.1200],
        [0.0170, 0.6928],
        [0.3637, 0.5090],
        [0.8377, 0.1941]])

In [90]:
logits = model(X)
print(logits.shape)
logits

torch.Size([5, 13])


tensor([[-3.2493e-01,  4.1555e-01,  6.5495e-01,  2.5975e-01, -5.7697e-01,
          1.0018e-01, -5.4894e-01,  4.6940e-01,  4.5887e-01,  9.4235e-02,
         -1.6835e-01,  4.7945e-01, -1.9655e-02],
        [-3.4495e-01,  3.9505e-01,  6.7346e-01,  2.7569e-01, -5.9716e-01,
          7.9201e-02, -5.5289e-01,  4.8960e-01,  4.2869e-01,  7.0087e-02,
         -1.9285e-01,  4.7948e-01, -4.1900e-04],
        [-3.5165e-01,  3.8818e-01,  6.7966e-01,  2.8103e-01, -6.0393e-01,
          7.2171e-02, -5.5421e-01,  4.9636e-01,  4.1858e-01,  6.1995e-02,
         -2.0106e-01,  4.7949e-01,  6.0272e-03],
        [-3.4723e-01,  3.9271e-01,  6.7557e-01,  2.7751e-01, -5.9947e-01,
          7.6807e-02, -5.5334e-01,  4.9190e-01,  4.2525e-01,  6.7331e-02,
         -1.9564e-01,  4.7948e-01,  1.7761e-03],
        [-3.4460e-01,  3.9540e-01,  6.7314e-01,  2.7541e-01, -5.9681e-01,
          7.9567e-02, -5.5282e-01,  4.8924e-01,  4.2922e-01,  7.0508e-02,
         -1.9242e-01,  4.7948e-01, -7.5419e-04]], grad_fn=<Addmm

In [91]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0468, 0.0982, 0.1248, 0.0841, 0.0364, 0.0717, 0.0374, 0.1037, 0.1026,
         0.0712, 0.0548, 0.1047, 0.0636],
        [0.0461, 0.0967, 0.1277, 0.0858, 0.0358, 0.0705, 0.0375, 0.1062, 0.1000,
         0.0698, 0.0537, 0.1052, 0.0651],
        [0.0459, 0.0961, 0.1287, 0.0864, 0.0356, 0.0701, 0.0375, 0.1071, 0.0991,
         0.0694, 0.0533, 0.1053, 0.0656],
        [0.0460, 0.0965, 0.1280, 0.0860, 0.0358, 0.0703, 0.0375, 0.1065, 0.0997,
         0.0697, 0.0536, 0.1052, 0.0653],
        [0.0461, 0.0967, 0.1276, 0.0858, 0.0358, 0.0705, 0.0375, 0.1062, 0.1000,
         0.0699, 0.0537, 0.1052, 0.0651]], grad_fn=<SoftmaxBackward0>)

In [92]:
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2, 2, 2, 2, 2])
